#### 6.12.목 RAG 수행
- 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템
- 콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
UPSTAGE_API_KEY= os.getenv("OPENAI_API_KEY")
print(UPSTAGE_API_KEY[:2])

sk


In [11]:
import os
from langchain_openai.embeddings import OpenAIEmbeddings  # OpenAI 임베딩 사용
from langchain.text_splitter import RecursiveCharacterTextSplitter  # 텍스트 분할기
from langchain_community.vectorstores import FAISS  # 벡터 DB (FAISS) 사용
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# PDF 파일 경로 설정
pdf_filepath = '../data/콘텐츠분쟁해결_사례.pdf'

# 파일 존재 여부 확인 (파일이 없으면 오류 발생)
if not os.path.exists(pdf_filepath):
    raise FileNotFoundError(f"파일을 찾을 수 없습니다: {pdf_filepath}")


# PDF 파일로드
loader = PyPDFLoader(pdf_filepath)
docs = loader.load()
# 총 페이지 로드
print(f"총 {len(docs)}페이지 로드 완료", type(docs[0]))

# 1단계 : 문서 분할 설정
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)

chunks = pythontext_splitter.split_documents(docs)

#2단계: 임베딩 모델 설정 
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)

# FAISS 백터 스토어에 저장하기
vectorstore = FAISS.from_documents(chunks, pythonembeddings)

#3단계: 검색기 설정 
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)
print(" Retriever 설정 완료")


#4단계: LLM 설정 
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)

#5단계: 법률 자문 프롬프트 작성 
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

# 프롬프트 객체화
prompt = PromptTemplate(
    template=pythonprompt_template,
    input_variables=["context", "question"]
)

#6단계: QA 체인 생성 
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)

#7단계: 테스트 질문 작성 
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

#8단계: 분쟁 유형 분류 함수 ( 선택 )
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


# 질문
for i, question in enumerate(pythontest_questions, 1):
    print(f" 질문 {i}/5 : {question}")

    dispute_type = classify_dispute_type(question)
    print(f" 분쟁 유형: {dispute_type}")

    print(" 답변 생성 중...")
    
    # RAG 실행
    full_question = f"[{dispute_type}] {question}"
    result = pythonqa_chain.invoke({"query": full_question})
    #result = pythonqa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)

총 109페이지 로드 완료 <class 'langchain_core.documents.base.Document'>
 Retriever 설정 완료
 질문 1/5 : 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?
 분쟁 유형: 게임
 답변 생성 중...

 답변:
--------------------------------------------------
온라인 게임에서 시스템 오류로 인해 아이템이 사라진 경우, 게임회사가 복구를 거부하는 상황에 대한 법률 조언을 다음과 같이 제공합니다.

1. **사례 기반 답변**:
   - 제시된 사례 중 2006년 사례에서는 시스템 오류로 아이템이 소멸된 경우에 대해 게임 회사가 계정 명의자가 아니면 복구 처리가 불가하다고 안내하였습니다. 이는 계정 명의와 관련된 문제로, 계정 명의자가 직접 복구 신청을 해야 한다는 점을 강조하고 있습니다.
   - 2009년 사례에서는 시스템 오류로 인한 게임머니 소실에 대해 시스템 오류가 발생하지 않았다고 판단하여 복구를 거부한 사례가 있습니다. 이는 시스템 오류의 발생 여부를 입증하는 것이 중요하다는 점을 보여줍니다.

2. **법적 근거 제시**:
   - 일반적으로 온라인 게임의 이용약관은 계약의 일부로 간주되며, 이용약관에 명시된 내용에 따라 분쟁이 해결됩니다. 따라서 이용약관을 꼼꼼히 검토하여 시스템 오류에 대한 복구 정책이 어떻게 규정되어 있는지 확인해야 합니다.

3. **판례 참조**:
   - 2008년 사례에서는 보안 서비스 미작동으로 인한 아이템 도난에 대해 게임사가 복구를 해준 사례가 있습니다. 이는 게임사의 서비스 미작동이 인정된 경우 복구가 가능하다는 점을 시사합니다.

4. **실무 가이드**:
   - **이용약관 검토**: 먼저 게임사의 이용약관을 검토하여 시스템 오류로 인한 아이템 소실에 대한 복구 정책이 어떻게 규정되어 있는지 확인합니다.
   - **증거 수집**: 시스템 오류가 발생